**About** : This notebook is used to generate PLs.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
torch.cuda.get_device_name(0)

In [ ]:
import os
import re
import cv2
import sys
import glob
import json
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
from utils.logger import Config, upload_to_kaggle

from params import *
from data.dataset import BreastDataset
from data.preparation import *
from data.transforms import get_transfos

from model_zoo.models import define_model
from utils.metrics import *
from utils.torch import load_model_weights
from utils.plots import plot_confusion_matrix

from inference.predict import predict, predict_tta
from inference.main import *

### Expes

In [ ]:
EXP_FOLDERS = [  # 1536x1024   0.5102  -  th=0.20  -  auc 0.9263  -  pct=98.367   LB 0.61
#     "../logs/2023-02-09/13/",  # nfl1 - 1536x1024  LR=3e-4  5ep gem CBIS           0.4734  -  th=0.23  -  auc 0.9141
    "../logs/2023-02-10/1/",  # nfl1 - 1536x1024  LR=3e-4  6ep gem CBIS           0.4756  -  th=0.20  -  auc 0.9127
    "../logs/2023-02-10/2/",  # nfl1 - 1536x1024  LR=4e-4  5ep gem CBIS CMMD      0.4703  -  th=0.19  -  auc 0.9125
    "../logs/2023-02-10/5/",  # v2-s - 1536x1024  LR=3e-4  5ep gem CBIS           0.4701  -  th=0.24  -  auc 0.9182
    "../logs/2023-02-10/6/",   # v2-s - 1536x1024  LR=4e-4  5ep gem CBIS           0.4771  -  th=0.27  -  auc 0.9190
]

In [ ]:
# EXP_FOLDERS = [  # 1536x1024   0.5401  -  th=0.21  -  auc 0.9317  -  pct=98.51
#     "../logs/2023-02-10/1/",   # nfl1 - 1536x1024  LR=3e-4  6ep gem CBIS           0.4756  -  th=0.20  -  auc 0.9127
#     "../logs/2023-02-10/6/",   # v2-s - 1536x1024  LR=4e-4  5ep gem CBIS           0.4771  -  th=0.27  -  auc 0.9190
#     "../logs/2023-02-10/16/",  # v2-m - 1536x1024  LR=3e-4  5ep bs6 CBIS CMMD PASM  0.4858  -  th=0.23  -  auc 0.9152
#     "../logs/2023-02-11/1/",   # nfl2 - 1536x1024  LR=4e-4  5ep bs8 CBIS            0.4881  -  th=0.21  -  auc 0.8994
#     "../logs/2023-02-11/3/",   # nfl2 - 1536x1024  LR=3e-4  5ep bs8 CBIS CMMD PASM  0.4837  -  th=0.25  -  auc 0.9106
# ]

### Main

In [ ]:
EXP_FOLDER = EXP_FOLDERS[0]

In [ ]:
DEBUG = False
SAVE = True
USE_FP16 = True

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))

In [ ]:
df = prepare_vindr_data(DATA_PATH, DATA_PATH + "vindr_" + config.img_folder)

In [ ]:
import os
import json
import glob
import numpy as np
import pandas as pd

from data.dataset import BreastDataset
from data.transforms import get_transfos
from model_zoo.models import define_model
from utils.logger import Config
from utils.metrics import tweak_thresholds
from utils.torch import load_model_weights
from inference.predict import predict, predict_tta


def kfold_inference_pl(df, exp_folder, debug=False, save=True, use_fp16=False):
    config = Config(json.load(open(exp_folder + "config.json", "r")))

    model = define_model(
        config.name,
        num_classes=config.num_classes,
        num_classes_aux=config.num_classes_aux,
        n_channels=config.n_channels,
        reduce_stride=config.reduce_stride,
        use_gem=config.use_gem,
        pretrained=False,
    )
    model = model.cuda().eval()

    weights = [f for f in sorted(glob.glob(exp_folder + "*.pt")) if "fullfit" not in f]
    folds = [int(w[-4]) for w in weights]

    save_folder = ""
    if save:
        save_folder = exp_folder + "pl/"
        os.makedirs(save_folder, exist_ok=True)

    for fold in folds:
        print(f"\n- Fold {fold + 1}")

        df_val = df.copy()
        if debug:
            df_val = df_val.head(4)

        dataset = BreastDataset(
            df_val,
            transforms=get_transfos(augment=False, resize=config.resize),
        )

        weight = weights[folds.index(fold)]
        model = load_model_weights(model, weight, verbose=1)

        pred_val, _ = predict(
            model,
            dataset,
            config.loss_config,
            batch_size=config.data_config["val_bs"],
            use_fp16=use_fp16,
        )

        if debug:
            return pred_val

        if save:
            np.save(save_folder + f"pred_pl_{fold}.npy", pred_val)

In [ ]:
for exp_folder in tqdm(EXP_FOLDERS):
    pred = kfold_inference_pl(
        df, exp_folder, debug=DEBUG, save=SAVE,use_fp16=USE_FP16
    )

### Results

In [ ]:
for fold in range(4):
    preds = []
    for exp_folder in EXP_FOLDERS:
        preds.append(np.load(exp_folder + f"pl/pred_pl_{fold}.npy"))
    df[f'pred_pl_{fold}'] = np.mean(preds, 0)

In [ ]:
df['pred_pl_fullfit'] = df[['pred_pl_0', 'pred_pl_1', 'pred_pl_2', 'pred_pl_3']].values.mean(-1)

In [ ]:
df.to_csv('../input/vindr_pl.csv', index=False)

In [ ]:
df[['BIRADS', 'pred_pl_fullfit']].groupby('BIRADS').mean()

In [ ]:
df[['BIRADS', 'pred_pl_fullfit']].groupby('BIRADS').max()

In [ ]:
# df_pl = prepare_pl_data(DATA_PATH, "vindr_yolox_1536_1024/", fold="fullfit")

Done ! 